In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

In [3]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model_4outputs_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_2_4_logits.keras"

In [4]:
model = keras.models.load_model(model_path)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
model_4outputs = keras.models.load_model(model_4outputs_path)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
model_4outputs.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,099 (301.17 KB)

 Trainable params: 25,699 (100.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 51,400 (200.78 KB)

In [7]:
decomon.__path__

['/home/aws_install/robustess_project/decomon/decomon/src/decomon']

In [8]:
from decomon.layers import DecomonLayer
from decomon.models import clone
from lipschitz_custom_tools import affine_bound_groupsort_output_keras, affine_bound_sqrt_output_keras, affine_bound_square_output_keras
from decomon.perturbation_domain import BallDomain

In [9]:
import pdb

In [10]:
class DecomonGroupSort2(DecomonLayer):
    layer : GroupSort2
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_groupsort_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up

In [11]:
model_4outputs.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2 (GroupSort2)        │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_sort2_1 (GroupSort2)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,099 (301.17 KB)

 Trainable params: 25,699 (100.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 51,400 (200.78 KB)

In [12]:
perturbation_domain = BallDomain(eps=0.1, p=2)

In [13]:
decomon_model = clone(model_4outputs, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=True, final_affine=False, perturbation_domain=perturbation_domain)

2025-04-24 17:31:48.683593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745508708.703535   24842 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745508708.710292   24842 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 17:31:48.730541: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
decomon_model.inputs

[<KerasTensor shape=(None, 1, 28, 28), dtype=float32, sparse=False, ragged=False, name=perturbation_domain_input_sequential>]

In [15]:
model_4outputs.inputs

[<KerasTensor shape=(None, 1, 28, 28), dtype=float32, sparse=False, ragged=False, name=input_layer_1>]

In [16]:
box = np.zeros((1,2,1,28,28))

In [17]:
box[:,1] += 0.1

In [20]:
decomon_model.predict(box)

ValueError: Input 0 of layer "decomon_model" is incompatible with the layer: expected shape=(None, 1, 28, 28), found shape=(1, 2, 1, 28, 28)

In [ ]:
class Sqrt(keras.layers.Layer):
    def call(self, inputs):
        return K.sqrt(inputs)

class Square(keras.layers.Layer):
    def call(self, inputs):
        return K.square(inputs)

In [ ]:
class DecomonSqrt(DecomonLayer):
    layer : Sqrt
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_sqrt_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up
class DecomonSquare(DecomonLayer):
    layer : Square
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_square_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up
    def forward_ibp_propagate(self, lower, upper):
        return self.layer(lower), self.layer(upper)

In [ ]:
sqrt = Sqrt()

In [ ]:
sqrt.predict(4)

AttributeError: 'Sqrt' object has no attribute 'predict'